In [ ]:
import tkinter as tk
from tkinter import messagebox
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense
from transformers import BertTokenizer, TFBertForSequenceClassification
import numpy as np
import tensorflow as tf

# Charger les embeddings GloVe
def load_glove_embeddings(file_path):
    print("Chargement des embeddings GloVe...")
    embeddings_index = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    print("Embeddings GloVe chargés.")
    return embeddings_index

# Appliquer les embeddings GloVe
def get_embedding_vector(tokens, embeddings_index):
    embedding_matrix = np.zeros((len(tokens), 100))  # 100 est la taille du vecteur GloVe
    valid_tokens = 0  # Compteur pour les tokens valides trouvés dans les embeddings GloVe
    
    for i, word in enumerate(tokens):
        embedding_vector = embeddings_index.get(word, None)
        if embedding_vector is not None:  # Si le mot est trouvé dans l'index GloVe
            embedding_matrix[i] = embedding_vector
            valid_tokens += 1
    
    if valid_tokens > 0:
        return np.mean(embedding_matrix[:valid_tokens], axis=0)
    else:
        return np.zeros(100)  # Si aucun mot valide n'a été trouvé, retourner un vecteur nul

# Charger le dataset
print("Chargement du dataset...")
data = pd.read_csv("E:/sentement140/training.1600000.processed.noemoticon.csv", encoding='ISO-8859-1', header=None)
data.columns = ["target", "id", "date", "flag", "user", "text"]
print("Dataset chargé avec succès.")

# Sélectionner les colonnes pertinentes
data = data[["target", "text"]]

# Convertir les valeurs cibles en sentiments
data['sentiment'] = data['target'].replace({0: "negative", 4: "positive", 2: "neutral"})
print("Conversion des valeurs cibles en sentiments effectuée.")

# Fonction pour nettoyer le texte
def clean_text(text):
    text = re.sub(r'http\S+', '', text)     # Supprimer les URLs
    text = re.sub(r'@\w+', '', text)        # Supprimer les mentions
    text = re.sub(r'#\w+', '', text)        # Supprimer les hashtags
    text = re.sub(r'[^\w\s]', '', text)     # Supprimer les caractères spéciaux
    text = text.lower()                     # Mettre tout en minuscules
    return text

# Appliquer le nettoyage
print("Nettoyage du texte...")
data['cleaned_text'] = data['text'].apply(clean_text)
print("Nettoyage du texte effectué.")

# Charger les embeddings GloVe
embeddings_index = load_glove_embeddings('E:/sentement140/glove.6B.100d.txt')

# Appliquer les embeddings GloVe aux tweets
data['tweet_embedding'] = data['cleaned_text'].apply(lambda x: get_embedding_vector(x.split(), embeddings_index))

# Fonction pour analyser le sentiment avec VADER
def analyze_vader_sentiment(text):
    sid = SentimentIntensityAnalyzer()
    sentiment_score = sid.polarity_scores(text)['compound']
    if sentiment_score >= 0.05:
        return "Positif", sentiment_score
    elif sentiment_score <= -0.05:
        return "Négatif", sentiment_score
    else:
        return "Neutre", sentiment_score

# Fonction pour analyser le sentiment avec Logistic Regression
def analyze_logistic_regression_sentiment(text, vectorizer, model):
    text_vect = vectorizer.transform([text])
    sentiment = model.predict(text_vect)[0]
    if sentiment == 0:
        return "Négatif"
    elif sentiment == 1:
        return "Positif"
    else:
        return "Neutre"

# Fonction pour analyser le sentiment avec LSTM
def analyze_lstm_sentiment(text, tokenizer, model):
    text_seq = tokenizer.texts_to_sequences([text])
    text_pad = pad_sequences(text_seq, maxlen=100)
    sentiment = (model.predict(text_pad) > 0.5).astype("int32")[0][0]
    if sentiment == 0:
        return "Négatif"
    elif sentiment == 1:
        return "Positif"
    else:
        return "Neutre"

# Fonction pour analyser le sentiment avec BERT
def analyze_bert_sentiment(text, tokenizer_bert, model):
    encodings = tokenizer_bert([text], truncation=True, padding=True, max_length=128, return_tensors="tf")
    sentiment = np.argmax(model.predict(encodings['input_ids']).logits, axis=1)[0]
    if sentiment == 0:
        return "Négatif"
    elif sentiment == 1:
        return "Positif"
    else:
        return "Neutre"

# Interface utilisateur avec Tkinter
window = tk.Tk()
window.title("Analyse de Sentiment")

# Définir la taille de la fenêtre
window.geometry("500x400")

# Ajouter un label pour les instructions
instructions_label = tk.Label(window, text="Entrez un tweet pour analyser le sentiment:")
instructions_label.pack(pady=10)

# Champ de texte pour entrer le tweet
entry_text = tk.Entry(window, width=50)
entry_text.pack(pady=10)

# Affichage des résultats
result_label = tk.Label(window, text="Résultat: ", font=("Arial", 12))
result_label.pack(pady=20)

# Bouton pour lancer l'analyse de sentiment avec VADER
def analyze():
    text = entry_text.get()
    
    if text:
        # Analyser le sentiment avec VADER
        sentiment_vader, score_vader = analyze_vader_sentiment(text)
        
        # Afficher les résultats dans l'interface
        result_label.config(text=f"VADER Sentiment: {sentiment_vader} (Score: {score_vader:.4f})")
    else:
        result_label.config(text="Veuillez entrer un tweet!")

# Bouton pour analyser avec VADER
analyze_button = tk.Button(window, text="Analyser avec VADER", command=analyze)
analyze_button.pack(pady=10)

# Fonction pour analyser avec Logistic Regression
def analyze_lr():
    text = entry_text.get()
    
    if text:
        sentiment_lr = analyze_logistic_regression_sentiment(text, vectorizer, log_reg)
        result_label.config(text=f"Logistic Regression Sentiment: {sentiment_lr}")
    else:
        result_label.config(text="Veuillez entrer un tweet!")

# Bouton pour analyser avec Logistic Regression
analyze_lr_button = tk.Button(window, text="Analyser avec Logistic Regression", command=analyze_lr)
analyze_lr_button.pack(pady=10)

# Fonction pour analyser avec LSTM
def analyze_lstm_func():
    text = entry_text.get()
    
    if text:
        sentiment_lstm = analyze_lstm_sentiment(text, tokenizer, lstm_model)
        result_label.config(text=f"LSTM Sentiment: {sentiment_lstm}")
    else:
        result_label.config(text="Veuillez entrer un tweet!")

# Bouton pour analyser avec LSTM
analyze_lstm_button = tk.Button(window, text="Analyser avec LSTM", command=analyze_lstm_func)
analyze_lstm_button.pack(pady=10)

# Fonction pour analyser avec BERT
def analyze_bert_func():
    text = entry_text.get()
    
    if text:
        sentiment_bert = analyze_bert_sentiment(text, tokenizer_bert, bert_model)
        result_label.config(text=f"BERT Sentiment: {sentiment_bert}")
    else:
        result_label.config(text="Veuillez entrer un tweet!")

# Bouton pour analyser avec BERT
analyze_bert_button = tk.Button(window, text="Analyser avec BERT", command=analyze_bert_func)
analyze_bert_button.pack(pady=10)

# Lancer la boucle principale de Tkinter
window.mainloop()


Chargement du dataset...
Dataset chargé avec succès.
Conversion des valeurs cibles en sentiments effectuée.
Nettoyage du texte...
Nettoyage du texte effectué.
Chargement des embeddings GloVe...
Embeddings GloVe chargés.


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Lenovo\anaconda3\envs\sentiment_analysis\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\Lenovo\AppData\Local\Temp\ipykernel_9840\2600290508.py", line 165, in analyze_lr
    sentiment_lr = analyze_logistic_regression_sentiment(text, vectorizer, log_reg)
NameError: name 'vectorizer' is not defined
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Lenovo\anaconda3\envs\sentiment_analysis\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\Lenovo\AppData\Local\Temp\ipykernel_9840\2600290508.py", line 165, in analyze_lr
    sentiment_lr = analyze_logistic_regression_sentiment(text, vectorizer, log_reg)
NameError: name 'vectorizer' is not defined
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Lenovo\anaconda3\envs\sentiment_analysis\lib\tkinter\__init__.py",